In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt 
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point,MultiPolygon,Polygon


import math

This is where the crs comes from: https://en.wikipedia.org/wiki/EPSG_Geodetic_Parameter_Dataset#:~:text=EPSG%3A4326%20%2D%20WGS%2084%2C,including%20Google%20Maps%20and%20OpenStreetMap.

In [ ]:
# county visualization shape file

gdf = gpd.read_file("cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
#print(gdf.crs)
gdf.to_crs('EPSG:4326') # converts to WSG84
gdf.head()

In [ ]:
# Land that people can hunt on 

huntingland = gpd.read_file("FWS_National_2022_-_2023_Hunt_Units/FWS_NWRS_HQ_PubHuntUnits.shp")
#print(huntingland['geometry'].crs) # this checks the gemoetry type
huntingland = huntingland.to_crs('EPSG:4326') # converts to WSG84
huntingland = huntingland[huntingland['Huntable']=='Yes']
huntingland.head()

In [ ]:
# First visualization

fig, ax = plt.subplots()

# points_gdf.plot(column='org_type', categorical=True, ax=ax, cmap='Set1', 
#                 zorder=3, marker = '.', markersize = 1, edgecolor = 'none',
#                legend=True, legend_kwds={'bbox_to_anchor':(.175,.35),'fontsize':4,'frameon':False,'markerscale':.5}
#                ) # these are the points

huntingland.plot(zorder = 2, ax = ax)

gdf.plot(column = 'STATEFP' ,color = 'white' , edgecolor = 'black',ax = ax, zorder = 1, linewidth=.05) # this is the US map


ax.axis('off')
ax.set_title('Food Points in USA',fontsize=12)


ax.set_xlim(-128, -60)
#ax.get_ylim()[1] #- use this for the full y axis
ax.set_ylim(22,50)
#plt.savefig('US_MAP.jpg',dpi = 4800)

In [ ]:
# stored as  EPSG:4326
water_gdf = gpd.read_file('Watershapefile/hydropol.shp')
water_gdf.head()

In [ ]:
# for waterbody in water_gdf['DESCRIPT'].unique():
#     print(waterbody)

In [ ]:
water_gdf['DESCRIPT'] = water_gdf['DESCRIPT'] .fillna('.')

# codes for subsettting the data
water_gdf['RIVER'] = water_gdf.apply(lambda x: 1 if x['DESCRIPT'][-5:]=='RIVER' else 0,axis = 1)
water_gdf['OCEAN'] = water_gdf.apply(lambda x: 1 if x['DESCRIPT'][-5:]=='OCEAN' else 0,axis = 1)
water_gdf['CHANNEL'] = water_gdf.apply(lambda x: 1 if x['DESCRIPT'][-7:]=='CHANNEL' else 0,axis = 1)
water_gdf['SEA'] = water_gdf.apply(lambda x: 1 if x['DESCRIPT'][-3:]=='SEA' else 0,axis = 1)
water_gdf['CREEK'] = water_gdf.apply(lambda x: 1 if x['DESCRIPT'][-4:]=='CREEK' else 0,axis = 1)



# complex geometry tag
water_gdf['COMPLEX'] = water_gdf[['RIVER','OCEAN','SEA','CHANNEL','CREEK']].sum(axis = 1) 
# convert to meters
water_gdf = water_gdf.to_crs("EPSG:3857") # Web mercator 84 ,like wsg 84 but the projection version
# susbet the data
# complex_water_gdf = water_gdf[water_gdf['COMPLEX']>0].copy()
# simple_water_gdf = water_gdf[water_gdf['COMPLEX']==0].copy()
# this subset the data into 500 meter increments instead of 40 feet
# complex_water_gdf['geometry_simple'] = complex_water_gdf['geometry'].apply(lambda x:
#                                                                            x.simplify(tolerance = 500, preserve_topology = False)
#                                                                           ).to_crs("EPSG:4326")

# simple_water_gdf['geometry_simple'] = simple_water_gdf['geometry'].centroid.to_crs("EPSG:4326")

# water_gdf = pd.concat([simple_water_gdf,complex_water_gdf], axis = 0)
# water_gdf = water_gdf.to_crs("EPSG:4326") # back to lat and long



water_gdf['geometry_simple'] = water_gdf['geometry'].apply(lambda x:
                                                           x.simplify(tolerance = 1000, preserve_topology = False)
                                                          ).to_crs("EPSG:4326")
                                                                           
                                                                           
water_gdf.head()

In [ ]:
# Different game available on the land

gametype = pd.read_csv('FWS_National_2022_-_2023_Hunt_Units.csv')
gametype.head()

In [ ]:
keep_game = ['Elk','Dark Geese', 'Dove', 'Light Geese',
            'Quail', 'Squirrel','Rabbit & Hare','Coot', 'Duck',
            'Feral Hog', 'Teal', 'White-Tailed Deer', 'Gallinule', 'Sea Duck',
            'Turkey', 'Moorhen', 'Swan', 'Grouse', 'Pronghorn',
            'Pheasant', 'Bear', 'Partridge', 'Sika Deer',
            'Mule Deer', 'Sandhill Crane', 'Moose', 'Caribou', 'Javelina',
            'Mountain Lion', 'Cormorant',
            'Prairie Dog', 'Pigeon', 'Bighorn Sheep',
            'Greater Prairie Chicken', 'Chukar', 'Alligator',
            'Nilgai Antelope', 'Mountain Goat', 'Fallow Deer',
            'Black-Tailed Deer', 'Russian Boar', 'Bison',
            'Dall Sheep', 'Oryx', 'Muskox']
keep_game = set(keep_game)
gametype['usable_game'] = gametype.apply(lambda x: 1 if x['Species'] in keep_game else 0,axis = 1)
gametype = gametype[['OBJECTID','usable_game']]
gametype = gametype[gametype['usable_game']>0]

In [ ]:
# This gives us census tract data from the ACS

census_tract = pd.read_csv('census_tract.csv', encoding="latin-1")
cols = list(census_tract.columns)
keep_cols = []
i = 0
for col in cols:
    if col[0:8] == 'Estimate':
        keep_cols.append(col)
    elif i == 0:
        keep_cols.append(col)
    elif i == 1:
        keep_cols.append(col)
    i+=1
del cols
census_tract = census_tract[keep_cols]
#census_tract.head()

In [ ]:
# this is the data I want to use
data_list =['Geography',
            'Geographic Area Name',
            'Estimate!!Total!!Total population',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!American Indian and Alaska Native',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Asian',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Native Hawaiian and Other Pacific Islander',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Some other race',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!Hispanic or Latino origin (of any race)',
            'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino',
            'Estimate!!Total!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population for whom poverty status is determined',
            'Estimate!!Total!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population for whom poverty status is determined!!Below 100 percent of the poverty level',
            'Estimate!!Total!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population for whom poverty status is determined!!100 to 149 percent of the poverty level',
            'Estimate!!Total!!POVERTY STATUS IN THE PAST 12 MONTHS!!Population for whom poverty status is determined!!At or above 150 percent of the poverty level']


col_list = ['Geography','Geographic Area Name', 'population','White','Black',
            'Native American','Asian','Hawaiian Pacific Islander', 'Other Race',
            'Two or More','Hispanic','White not Hispanic','povertystatus_denom','below_poverty', '100-% poverty',
            '150-% poverty', '150+% poverty']

conversion_dict = dict(zip(data_list, col_list))
census_tract = census_tract[data_list]
census_tract = census_tract.rename(mapper = conversion_dict, axis = 1)

In [ ]:
# I am doing a very evil thing here 
def newstring(string):
    return(string[9:])
census_tract['GEO_ID'] = census_tract['Geography'].apply(lambda x: newstring(x))
def newstring(string):
    return(string[0:2])
FIPS = census_tract['GEO_ID'].apply(lambda x: newstring(x))
FIPS = FIPS.unique()

In [ ]:
census_tract.head()

In [ ]:
""" this is how I construct the wgts between 0 and 1, it depends evenly '
    on the percent black native hawaiian other t and belowe 100% of the poverty line"""

names = ['Black','Native American','Hawaiian Pacific Islander',
         'Other Race','Two or More',"Hispanic",'below_poverty']

census_tract[names] = census_tract[names].apply(pd.to_numeric, errors='coerce', axis=1)
census_tract['wgt'] = census_tract[names].sum(axis = 1)
census_tract['county_id'] = census_tract['GEO_ID'].apply(lambda x: x[0:5])
census_county = census_tract.groupby('county_id').sum().reset_index()
census_county = census_county[['wgt','county_id']].rename({'wgt':'wgt_sum'},axis = 1)
census_tract = census_tract.merge(census_county, on = 'county_id',how = 'left')
census_tract['wgt'] = census_tract['wgt']/census_tract['wgt_sum']

right_df = census_tract[['GEO_ID','county_id','wgt']]

Our center is the mean lat and longitude of all available points:

- share by race that is food insecure white black hispanic and other
- other is pi and native american

- 150\% of federal poverty line
- Percent of the county's population that is in the cnesus tract

In [ ]:
# this is how I get the shape files for census tract by county

censustract_gdflist = []
for fip in FIPS:
    
    url = 'Census Tract Shape Files/tl_2020_'+fip+'_tract/tl_2020_'+fip+'_tract.shp'

    ctract = gpd.read_file(url)
    ctract = ctract.to_crs("EPSG:3857") # Web mercator 84 like wsg 84 but the projection version
    ctract['centroid'] = ctract['geometry'].centroid
    ctract['lon'] = ctract.centroid.apply(lambda p: p.x)
    ctract['lat'] = ctract.centroid.apply(lambda p: p.y)
    censustract_gdflist.append(ctract)

In [ ]:
# Get the centroid of huntable land

huntingland_centroid = huntingland.to_crs("EPSG:3857") # Web mercator 84 like wsg 84 but the projection version
huntingland_centroid['centroid'] = huntingland_centroid['geometry'].centroid
huntingland_centroid['lon'] = huntingland_centroid.centroid.apply(lambda p: p.x)
huntingland_centroid['lat'] = huntingland_centroid.centroid.apply(lambda p: p.y)

huntingland_centroid = huntingland_centroid[['OBJECTID','lon','lat']]

In [ ]:
huntingland_centroid.head()

In [ ]:
#this is the weighted centroid
synth_family = pd.DataFrame(columns = ['county_id','lon_wgt','lat_wgt'])
for geodf in censustract_gdflist:
    geodf = geodf.merge(right_df, left_on = 'GEOID',right_on = 'GEO_ID')
    geodf[['lon_wgt','lat_wgt']] = geodf[['lon','lat']].multiply(geodf['wgt'],axis=0)
    geodf = geodf[['lon_wgt','lat_wgt','county_id']].groupby('county_id').sum().reset_index()
    synth_family = pd.concat([synth_family,geodf])
    

In [ ]:
# I lose some territories
synth_family = synth_family.dropna(axis = 0)
synth_family_gdf = gpd.GeoDataFrame(synth_family,
                                            geometry=gpd.points_from_xy(synth_family.lon_wgt, synth_family.lat_wgt),
                                            crs="EPSG:3857")
synth_family_gdf = synth_family_gdf.to_crs("EPSG:4326") # to degrees
#synth_family= synth_family.to_crs("EPSG:4269")


In [ ]:
csa_names = ['listing_id','location_x','location_y',
             'acceptedpayment','FNAP','products','product_fruit',
             'product_vegetables','product_fishseafood',
             'product_poultryfowl_otherdesc','product_poultryfowl','product_redmeat']

farmersmarket_names = ['listing_id','location_x','location_y','acceptedpayment','FNAP']

foodhubs_names = ['listing_id','location_x','location_y','acceptedpayment','FNAP','products']



farm_mark = pd.read_excel('PointData/FarmersMarket.xlsx')
farm_mark = farm_mark[farmersmarket_names]
farm_mark_gdf = gpd.GeoDataFrame(farm_mark, 
                                 geometry=gpd.points_from_xy(farm_mark.location_x, farm_mark.location_y),
                                 crs="EPSG:4269") # not sure but they are long and lat
farm_mark_gdf = farm_mark_gdf[farm_mark_gdf['geometry'].is_empty != True]

csa = pd.read_excel('PointData/CSA.xlsx')
csa = csa[csa_names]
csa_gdf = gpd.GeoDataFrame(csa,
                           geometry=gpd.points_from_xy(csa.location_x, csa.location_y),
                           crs="EPSG:4269")
csa_gdf = csa_gdf[csa_gdf['geometry'].is_empty != True]


foodhub = pd.read_excel('PointData/Foodhub.xlsx')
foodhub = foodhub[foodhubs_names]
foodhub_gdf = gpd.GeoDataFrame(foodhub,
                               geometry=gpd.points_from_xy(foodhub.location_x, foodhub.location_y),
                               crs="EPSG:4269")
foodhub_gdf = foodhub_gdf[foodhub_gdf['geometry'].is_empty != True]


del csa,foodhub,farm_mark

farm_mark_gdf = farm_mark_gdf.to_crs("EPSG:4326") # to degrees
foodhub_gdf = foodhub_gdf.to_crs("EPSG:4326")
csa_gdf = csa_gdf.to_crs("EPSG:4326")


In [ ]:
huntingland_centroid_df=pd.DataFrame(huntingland_centroid)
huntingland_centroid_gdf=gpd.GeoDataFrame(huntingland_centroid_df, 
                                          geometry = gpd.points_from_xy(huntingland_centroid_df.lon, huntingland_centroid_df.lat),
                                          crs = 'EPSG:3857')

huntingland_centroid_gdf = huntingland_centroid_gdf.to_crs("EPSG:4326")

In [ ]:
#Vegetables
fruit_veglist = ['product_vegetables','product_fruit']
for plant in fruit_veglist:
    csa_gdf[plant] = csa_gdf[plant].isnull()
    csa_gdf[plant] = csa_gdf.apply(lambda x: 1 if x[plant] == False else 0, axis = 1)

#Meatlist
meatlist = ['product_fishseafood','product_poultryfowl_otherdesc','product_poultryfowl','product_redmeat']
for meat in meatlist:
    csa_gdf[meat] = csa_gdf[meat].isnull()
    csa_gdf[meat] = csa_gdf.apply(lambda x: 1 if x[meat] == False else 0, axis = 1)

csa_gdf['plant'] = csa_gdf[fruit_veglist].sum(axis=1)
csa_gdf['meat'] = csa_gdf[meatlist].sum(axis=1)


In [ ]:
# this is for meat and plant 

cols = ['listing_id','geometry']

meat_gdf = csa_gdf[csa_gdf['meat']>=1]
plant_gdf = csa_gdf[csa_gdf['plant']>=1]

meat_gdf = pd.concat([meat_gdf[cols],farm_mark_gdf[cols]],axis = 0)
plant_gdf = pd.concat([plant_gdf[cols],farm_mark_gdf[cols]],axis = 0)


divide up point data into meat veg and foodhub if wic and/or snap, min distance by each category

In [ ]:
synth_family_df = pd.DataFrame(synth_family_gdf)

synth_family_df['county_id'] = pd.to_numeric(synth_family_df['county_id'])
synth_family_df = synth_family_df[synth_family_df['county_id']<72000]

meat_df = pd.DataFrame(meat_gdf)
foodhub_df = pd.DataFrame(foodhub_gdf)
plant_df = pd.DataFrame(plant_gdf)
huntingland_centroid_df = pd.DataFrame(huntingland_centroid_gdf)
huntingland_centroid_df = huntingland_centroid_df[['OBJECTID','geometry']]

In [ ]:
synth_family_df.to_excel('synthetic_family.xlsx')

In [ ]:
huntingland_centroid_df = huntingland_centroid_df.merge(gametype, 
                                                        how = 'left', on = 'OBJECTID',indicator = True)
huntingland_centroid_df =  huntingland_centroid_df[huntingland_centroid_df['_merge']=='both']
huntingland_centroid_df =  huntingland_centroid_df[['OBJECTID','geometry']]

In [ ]:
#Calculate distance using the Haversine Formula
def rad(deg):
    radians = deg * math.pi/180
    return(radians)

def haversine(coord1,coord2):
    
    lon1 = rad(coord1.x)
    lat1 = rad(coord1.y)
    lon2 = rad(coord2.x)
    lat2 = rad(coord2.y)

    r = 6371000.7900 
    
    delta_phi = lat2 - lat1
    delta_lambda = lon2 - lon1

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = r * c  # output distance in meters
    miles = meters * 0.0006213711922 # output distance in miles
    
    miles = round(miles, 2)


    return(miles)

def haversine_water(coord1,coord2):
    
    lon1 = rad(coord1.x)
    lat1 = rad(coord1.y)
    lon2 = rad(coord2[0])
    lat2 = rad(coord2[1])

    r = 6371000.7900 
    
    delta_phi = lat2 - lat1
    delta_lambda = lon2 - lon1

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = r * c  # output distance in meters
    miles = meters * 0.0006213711922 # output distance in miles
    
    miles = round(miles, 2)


    return(miles)

In [ ]:
def min_distance(df_list,synth_family_df,name_list):
    output_df = pd.DataFrame(synth_family_df['county_id'])
    i = 0
    for df in df_list:
        cross_prod = synth_family_df.merge(df, how = 'cross')
        cross_prod['distance'] = cross_prod.apply(lambda x: haversine(x['geometry_x'], x['geometry_y']), axis = 1)
        
        temp_df = cross_prod.groupby('county_id').min('distance').reset_index()
        temp_df = temp_df[['county_id','distance']]
        name = 'distance_'+name_list[i]
        temp_df = temp_df.rename({'distance':name},axis = 1)
        i+=1
        output_df = output_df.merge(temp_df, how = 'left', on = 'county_id')
        
    return(output_df)

In [ ]:
# #synth_family

fig, ax = plt.subplots()

synth_family_gdf.plot( ax=ax, zorder=2, marker = '.', markersize = .1,edgecolor = 'none') # these are the points

gdf.plot(column = 'STATEFP' ,color = 'white' , edgecolor = 'black',ax = ax, zorder = 1, linewidth=.05) # this is the US map

huntingland.plot(zorder = 3, ax = ax, markersize = .1, color = 'yellow')
meat_gdf.plot(zorder = 4, ax = ax, markersize = .1,edgecolor = 'none', color = 'red')
foodhub_gdf.plot(zorder = 5, ax = ax, markersize = .1,edgecolor = 'none', color = 'brown')
plant_gdf.plot(zorder = 6, ax = ax, markersize = .1,edgecolor = 'none', color = 'green')
water_gdf.plot(zorder = 7, ax = ax, markersize = .1,edgecolor = 'none', color = 'blue')


ax.axis('off')

ax.set_xlim(-128, -60)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(22,50)
#plt.savefig('synthetic_hh.jpg',dpi = 4800)

In [ ]:
# water dataframe substted

water_gdf['unit_id'] = range(0,len(water_gdf))
water_gdf = water_gdf[['unit_id','centroid','geometry']]

In [ ]:
def extract_boundary_points(geometry):
    boundary_points = []
    if isinstance(geometry, Polygon) == True:
        boundary_points = list(geometry.exterior.coords)
        
    elif isinstance(geometry, MultiPolygon) == True:
        for geom in geometry.geoms:
            boundary_points.extend(list(geom.exterior.coords))

    return(boundary_points)

In [ ]:
# this is how I extract the boundayr points

master_coord = []
for coordlist in list(water_gdf['geometry_simple'].apply(lambda x: extract_boundary_points(x))):
    master_coord.extend(coordlist)

In [ ]:
master_coord_df = pd.DataFrame(master_coord, columns = [' long','lat'])
master_coord_df['geometry_2'] = master_coord
master_coord_df = master_coord_df['geometry_2']
master_coord_df.head()

In [ ]:
synth_family_water_df = synth_family_df[['geometry','county_id']]

In [ ]:
water_compare_df = synth_family_water_df.merge(master_coord_df, how = 'cross')

In [ ]:
water_compare_df.head()

In [ ]:
water_compare_df['distance'] = water_compare_df.apply(lambda x: haversine_water(x['geometry'],x['geometry_2']),
                                                      axis = 1)

In [ ]:
water_min_diff = water_compare_df.groupby('county_id').min('distance')

In [ ]:
water_min_diff.to_csv('results_waterdata.csv')

In [ ]:
county_point_list = list(synth_family_df['geometry'])
county_name_list = list(synth_family_df['county_id'])

file = open('distances.txt','w')
county_min_list = []
i=0 # county indexer
for county in county_point_list:
    min_val = None # prime the minimum value
    for geometry in master_coord: # looking through all lakes, geometry is a coordninate
        val = haversine_water(county,geometry) # this is the value we compare
        if min_val == None: # first iteration check
            min_val = val # assign value
        elif min_val > val: # otherwise if val ie less thatn min val
            min_val = val # assign value
            
    if i % 200 == 0:
        print(i)
    county_min_list.append((county_name_list[i],min_val)) # append the minimum
    file.write(str(county_name_list[i])+','+str(min_val)+"\n")
    i+=1
file.close()

In [ ]:

df_list = [meat_df, plant_df, foodhub_df,huntingland_centroid_df]
name_list = ['meat','plant','foodhub','hunting']

location_df = min_distance(df_list,synth_family_df,name_list)

In [ ]:
location_df['lon'] = location_df['geometry'].apply(lambda x: x.coords.x,)
location_df.to_excel('county_distance.xlsx')

PCA component

In [ ]:
county_food_sec_df = pd.read_csv('variables_for_index_clean.csv')
county_food_sec_df = pd.pivot(county_food_sec_df, index='fips', columns='variable_name', values='value').reset_index()
county_food_sec_df= county_food_sec_df.dropna(axis = 0)
county_food_sec_df.head()

In [ ]:
location_df['county_id'] = pd.to_numeric(location_df['county_id'])
master_df = location_df.merge(county_food_sec_df,how = 'left', left_on ='county_id',right_on = 'fips' )

In [ ]:
master_df.to_excel('master_file.xlsx')

In [ ]:
# read stuff in

In [ ]:
master_df = pd.read_excel('master_file.xlsx')
master_df.head()

In [ ]:
gdf['county_id'] = gdf.apply(lambda x: str(x['STATEFP'])+str(x['COUNTYFP']),axis = 1)
gdf['county_id'] = pd.to_numeric(gdf['geoid'])

In [ ]:
food_sec_gdf= gdf.merge(index, how = 'left', on = 'county_id')

In [ ]:
# #synth_family

fig, ax = plt.subplots()

food_sec_gdf.plot(column = 'index' ,ax = ax,cmap = 'seismic',
                  zorder = 1, linewidth=.05,
                  legend = True ) # this is the US map

# huntingland.plot(zorder = 3, ax = ax, markersize = .1)
# meat_gdf.plot(zorder = 4, ax = ax, markersize = .1)
# foodhub_gdf.plot(zorder = 5, ax = ax, markersize = .1)
# plant_gdf.plot(zorder = 6, ax = ax, markersize = .1)



ax.axis('off')

ax.set_xlim(-128, -60)
#ax.get_ylim()[1] - use this for the full y axis
ax.set_ylim(22,50)
plt.savefig('INDEX_MAP.jpg',dpi = 4800)